In [19]:
!pip install psycopg2-binary

In [20]:
import requests
import pandas as pd
import io

In [21]:
url = 'https://data.sfgov.org/resource/wr8u-xric.json'

urlData = requests.get(url).content
df = pd.read_json(io.StringIO(urlData.decode('utf-8')))
print(df)

     incident_number  exposure_number        id                  address  \
0            8028304                0  80283040            150 Elsie St.   
1            8028303                0  80283030            85 Turner Tr.   
2            8028309                0  80283090              175 6th St.   
3            8028314                0  80283140            633 Hayes St.   
4            8028319                0  80283190  27th Av. / Cabrillo St.   
..               ...              ...       ...                      ...   
995          3004089                0  30040890            3315 20th St.   
996          3004091                0  30040910         680 Prentiss St.   
997          3004092                0  30040920         214 Van Ness Av.   
998          3004093                0  30040930          101 Blanken Av.   
999          3004094                0  30040940     21st Av. / Judah St.   

           incident_date  call_number           alarm_dttm  \
0    2008-04-01T00:00:00 

In [22]:
df.shape

(1000, 64)

In [23]:
len(df.columns.values), df.columns.values

(64,
 array(['incident_number', 'exposure_number', 'id', 'address',
        'incident_date', 'call_number', 'alarm_dttm', 'arrival_dttm',
        'close_dttm', 'city', 'zipcode', 'battalion', 'station_area',
        'suppression_units', 'suppression_personnel', 'ems_units',
        'ems_personnel', 'other_units', 'other_personnel',
        'first_unit_on_scene', 'fire_fatalities', 'fire_injuries',
        'civilian_fatalities', 'civilian_injuries', 'number_of_alarms',
        'primary_situation', 'mutual_aid', 'action_taken_primary',
        'action_taken_secondary', 'action_taken_other',
        'detector_alerted_occupants', 'property_use',
        'supervisor_district', 'neighborhood_district', 'point',
        'estimated_contents_loss', 'area_of_fire_origin', 'ignition_cause',
        'ignition_factor_primary', 'ignition_factor_secondary',
        'heat_source', 'item_first_ignited',
        'human_factors_associated_with_ignition',
        'estimated_property_loss', 'structure_type

## Building columns for facts and dimensions tables

In [24]:
incident_fact_columns = [
'incident_number',
'exposure_number',
'id',
'address',
'incident_date',
'call_number',
'alarm_dttm',
'arrival_dttm',
'close_dttm',
'city',
'zipcode',
'battalion',
'station_area',
'suppression_units',
'suppression_personnel',
'ems_units',
'ems_personnel',
'other_units',
'other_personnel',
'first_unit_on_scene',
'fire_fatalities',
'fire_injuries',
'civilian_fatalities',
'civilian_injuries',
'number_of_alarms',
'primary_situation',
'mutual_aid',
'action_taken_primary',
'action_taken_secondary',
'action_taken_other',
'detector_alerted_occupants',
'property_use',
'supervisor_district',
'neighborhood_district',
'point',
'estimated_contents_loss',
'area_of_fire_origin',
'ignition_cause',
'ignition_factor_primary',
'ignition_factor_secondary',
'heat_source',
'item_first_ignited',
'human_factors_associated_with_ignition',
'estimated_property_loss',
'structure_type',
'structure_status',
'floor_of_fire_origin',
'fire_spread',
'no_flame_spead',
'number_of_floors_with_minimum_damage',
'number_of_floors_with_significant_damage',
'number_of_floors_with_heavy_damage',
'number_of_floors_with_extreme_damage',
'detectors_present',
'detector_type',
'detector_operation',
'detector_effectiveness',
'detector_failure_reason',
'automatic_extinguishing_system_present',
'automatic_extinguishing_sytem_type',
'automatic_extinguishing_sytem_perfomance',
'automatic_extinguishing_sytem_failure_reason',
'number_of_sprinkler_heads_operating',
'box'
]

time_dimension_columns = ['incident_date',
'alarm_dttm',
'arrival_dttm',
'close_dttm']

identification_columns = ['id']

casualties_dimension_columns = [
    'fire_fatalities',
    'fire_injuries',
    'civilian_fatalities',
    'civilian_injuries'
]

location_dimension_columns = ['address', 'city', 'zipcode', 'supervisor_district', 'neighborhood_district', 'point']

new_columns = [col for col in incident_fact_columns if col not in location_dimension_columns and col not in time_dimension_columns and col not in casualties_dimension_columns]

# Foreign keys assigna

location_dimension_columns += identification_columns
time_dimension_columns += identification_columns
casualties_dimension_columns += identification_columns

new_columns, len(new_columns)

(['incident_number',
  'exposure_number',
  'id',
  'call_number',
  'battalion',
  'station_area',
  'suppression_units',
  'suppression_personnel',
  'ems_units',
  'ems_personnel',
  'other_units',
  'other_personnel',
  'first_unit_on_scene',
  'number_of_alarms',
  'primary_situation',
  'mutual_aid',
  'action_taken_primary',
  'action_taken_secondary',
  'action_taken_other',
  'detector_alerted_occupants',
  'property_use',
  'estimated_contents_loss',
  'area_of_fire_origin',
  'ignition_cause',
  'ignition_factor_primary',
  'ignition_factor_secondary',
  'heat_source',
  'item_first_ignited',
  'human_factors_associated_with_ignition',
  'estimated_property_loss',
  'structure_type',
  'structure_status',
  'floor_of_fire_origin',
  'fire_spread',
  'no_flame_spead',
  'number_of_floors_with_minimum_damage',
  'number_of_floors_with_significant_damage',
  'number_of_floors_with_heavy_damage',
  'number_of_floors_with_extreme_damage',
  'detectors_present',
  'detector_type',

In [25]:
incident_table = df[new_columns]
location_dimension = df[location_dimension_columns]
time_dimension = df[time_dimension_columns]
casualties_dimension = df[casualties_dimension_columns]

time_dimension = time_dimension.rename(columns={'id': 'incident_id'})
location_dimension = location_dimension.rename(columns={'id': 'incident_id'})
casualties_dimension = casualties_dimension.rename(columns={'id': 'incident_id'})

print('-----------')
print('incident_table')
print(incident_table.columns.values)
print('-----------')
print('location_dimension')
print(location_dimension.columns.values)
print('-----------')
print('time_dimension')
print(time_dimension.columns.values)
print('-----------')
print('casualties_dimension')
print(casualties_dimension.columns.values)

-----------
incident_table
['incident_number' 'exposure_number' 'id' 'call_number' 'battalion'
 'station_area' 'suppression_units' 'suppression_personnel' 'ems_units'
 'ems_personnel' 'other_units' 'other_personnel' 'first_unit_on_scene'
 'number_of_alarms' 'primary_situation' 'mutual_aid'
 'action_taken_primary' 'action_taken_secondary' 'action_taken_other'
 'detector_alerted_occupants' 'property_use' 'estimated_contents_loss'
 'area_of_fire_origin' 'ignition_cause' 'ignition_factor_primary'
 'ignition_factor_secondary' 'heat_source' 'item_first_ignited'
 'human_factors_associated_with_ignition' 'estimated_property_loss'
 'structure_type' 'structure_status' 'floor_of_fire_origin' 'fire_spread'
 'no_flame_spead' 'number_of_floors_with_minimum_damage'
 'number_of_floors_with_significant_damage'
 'number_of_floors_with_heavy_damage'
 'number_of_floors_with_extreme_damage' 'detectors_present'
 'detector_type' 'detector_operation' 'detector_effectiveness'
 'detector_failure_reason' 'automa

In [26]:
for col in incident_table.columns.values:
    #get dtype for column
    dt = incident_table[col].dtype
    #check if it is a number
    if dt == int or dt == 'int64' or dt == float:
        print('--------------')
        print(col, dt)
        incident_table[col] = incident_table[col].fillna(0)
    else:
        incident_table[col] = incident_table[col].fillna("-")

for col in time_dimension.columns.values:
    #get dtype for column
    dt = time_dimension[col].dtype 
    #check if it is a number
    if dt == int or dt == 'int64' or dt == float:
        time_dimension[col] = time_dimension[col].fillna(0)
    else:
        time_dimension[col] = time_dimension[col].fillna("-")

for col in location_dimension.columns.values:
    #get dtype for column
    dt = location_dimension[col].dtype 
    #check if it is a number
    if dt == int or dt == 'int64' or dt == float:
        location_dimension[col] = location_dimension[col].fillna(0)
    else:
        location_dimension[col] = location_dimension[col].fillna("-")

for col in casualties_dimension.columns.values:
    #get dtype for column
    dt = casualties_dimension[col].dtype 
    #check if it is a number
    if dt == int or dt == 'int64' or dt == float:
        casualties_dimension[col] = casualties_dimension[col].fillna(0)
    else:
        casualties_dimension[col] = casualties_dimension[col].fillna("-")

--------------
incident_number int64
--------------
exposure_number int64
--------------
id int64
--------------
call_number int64
--------------
suppression_units int64
--------------
suppression_personnel int64
--------------
ems_units int64
--------------
ems_personnel int64
--------------
other_units int64
--------------
other_personnel int64
--------------
number_of_alarms int64
--------------
estimated_contents_loss float64
--------------
estimated_property_loss float64
--------------
floor_of_fire_origin float64
--------------
no_flame_spead float64
--------------
number_of_floors_with_minimum_damage float64
--------------
number_of_floors_with_significant_damage float64
--------------
number_of_floors_with_heavy_damage float64
--------------
number_of_floors_with_extreme_damage float64
--------------
number_of_sprinkler_heads_operating float64
--------------
box float64


/tmp/ipykernel_180/3426958609.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_table[col] = incident_table[col].fillna(0)
/tmp/ipykernel_180/3426958609.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_table[col] = incident_table[col].fillna("-")


In [27]:
print(len(incident_table[incident_table['automatic_extinguishing_system_present'] == '1 -Present']))
incident_table['automatic_extinguishing_system_present'] = incident_table['automatic_extinguishing_system_present'].map(lambda x: x == '1 -Present')
incident_table[incident_table['automatic_extinguishing_system_present']]

4


/tmp/ipykernel_180/3400057629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_table['automatic_extinguishing_system_present'] = incident_table['automatic_extinguishing_system_present'].map(lambda x: x == '1 -Present')


,incident_number,exposure_number,id,call_number,battalion,station_area,suppression_units,suppression_personnel,ems_units,ems_personnel,...,detector_type,detector_operation,detector_effectiveness,detector_failure_reason,automatic_extinguishing_system_present,automatic_extinguishing_sytem_type,automatic_extinguishing_sytem_perfomance,automatic_extinguishing_sytem_failure_reason,number_of_sprinkler_heads_operating,box
127,3001340,0,30013400,30040349,B03,35,2,9,0,0,...,1 -Smoke,2 -Detector operated,"1 -Alerted occupants, occupants responded",-,True,-,-,-,0.0,0.0
288,3003360,0,30033600,30110387,B04,16,5,21,1,2,...,-,-,-,-,True,-,-,-,0.0,0.0
465,3006712,0,30067120,30230425,B03,03,8,30,0,0,...,1 -Smoke,2 -Detector operated,"1 -Alerted occupants, occupants responded",-,True,-,-,-,0.0,0.0
540,3002174,0,30021740,30070383,B03,01,9,33,0,0,...,-,-,-,-,True,-,-,-,0.0,0.0


In [28]:
incident_table['automatic_extinguishing_sytem_perfomance'].astype('category')

0      -
1      -
2      -
3      -
4      -
      ..
995    -
996    -
997    -
998    -
999    -
Name: automatic_extinguishing_sytem_perfomance, Length: 1000, dtype: category
Categories (1, object): ['-']

In [29]:
incident_table['call_number']

0      80920257
1      80920256
2      80920262
3      80920268
4      80920273
         ...   
995    30140267
996    30140271
997    30140276
998    30140277
999    30140278
Name: call_number, Length: 1000, dtype: int64

In [30]:
incident_table = incident_table.astype({
    'id': 'string',
    'call_number': 'string',
    'battalion': 'category',
    'station_area': 'category',
    'first_unit_on_scene': 'category',
    'primary_situation': 'category',
    'action_taken_primary': 'category',
    'action_taken_secondary': 'category',
    'action_taken_other': 'category',
    'detector_alerted_occupants': 'category',
    'property_use': 'category',
    'area_of_fire_origin': 'category',
    'ignition_cause': 'category',
    'ignition_factor_primary': 'category',
    'ignition_factor_secondary': 'category',
    'heat_source': 'category',
    'item_first_ignited': 'category',
    'human_factors_associated_with_ignition': 'category',
    'structure_type': 'category',
    'structure_status': 'category',
    'fire_spread': 'category',
    'detectors_present': 'category',
    'detector_type': 'category',
    'detector_operation': 'category',
    'detector_effectiveness': 'category',
    'detector_failure_reason': 'category',
    'automatic_extinguishing_sytem_type': 'category'
})

In [36]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres:yourpassword@db:5432/yourdb')


In [37]:
for col in incident_table:
    print(col, incident_table[col].dtype)

incident_number int64
exposure_number int64
id string
call_number string
battalion category
station_area category
suppression_units int64
suppression_personnel int64
ems_units int64
ems_personnel int64
other_units int64
other_personnel int64
first_unit_on_scene category
number_of_alarms int64
primary_situation category
mutual_aid object
action_taken_primary category
action_taken_secondary category
action_taken_other category
detector_alerted_occupants category
property_use category
estimated_contents_loss float64
area_of_fire_origin category
ignition_cause category
ignition_factor_primary category
ignition_factor_secondary category
heat_source category
item_first_ignited category
human_factors_associated_with_ignition category
estimated_property_loss float64
structure_type category
structure_status category
floor_of_fire_origin float64
fire_spread category
no_flame_spead float64
number_of_floors_with_minimum_damage float64
number_of_floors_with_significant_damage float64
number_of_floo

In [38]:
incident_table['mutual_aid']

0      None
1      None
2      None
3      None
4      None
       ... 
995    None
996    None
997    None
998    None
999    None
Name: mutual_aid, Length: 1000, dtype: object

In [39]:
print('-----------')
print('location_dimension')
for col in location_dimension:
    print(col, location_dimension[col].dtype)

location_dimension['point']

-----------
location_dimension
address object
city object
zipcode int64
supervisor_district float64
neighborhood_district object
point object
incident_id int64


0      {'type': 'Point', 'coordinates': [-122.4183733...
1      {'type': 'Point', 'coordinates': [-122.39489, ...
2      {'type': 'Point', 'coordinates': [-122.407468,...
3      {'type': 'Point', 'coordinates': [-122.4268490...
4      {'type': 'Point', 'coordinates': [-122.4863941...
                             ...                        
995    {'type': 'Point', 'coordinates': [-122.4150297...
996    {'type': 'Point', 'coordinates': [-122.4126249...
997    {'type': 'Point', 'coordinates': [-122.419656,...
998    {'type': 'Point', 'coordinates': [-122.400681,...
999    {'type': 'Point', 'coordinates': [-122.479138,...
Name: point, Length: 1000, dtype: object

In [41]:
engine = create_engine('postgresql://postgres:yourpassword@db:5432/yourdb')
location_dimension.to_sql('locations_DM', engine)
# thros error if the data has already been created

ValueError: Table 'locations_DM' already exists.

In [16]:
print('-----------')
print('casualties_dimension')
for col in casualties_dimension:
    print(col, casualties_dimension[col].dtype)

-----------
casualties_dimension
fire_fatalities int64
fire_injuries int64
civilian_fatalities int64
civilian_injuries int64
incident_id int64


In [17]:
print('-----------')
print('time_dimension')
for col in time_dimension:
    print(col, time_dimension[col].dtype)

-----------
time_dimension
incident_date object
alarm_dttm object
arrival_dttm object
close_dttm object
incident_id int64


In [18]:
incident_table[incident_table['primary_situation'] == '412 - Gas leak (natural gas or LPG)']

,incident_number,exposure_number,id,call_number,battalion,station_area,suppression_units,suppression_personnel,ems_units,ems_personnel,...,detector_type,detector_operation,detector_effectiveness,detector_failure_reason,automatic_extinguishing_system_present,automatic_extinguishing_sytem_type,automatic_extinguishing_sytem_perfomance,automatic_extinguishing_sytem_failure_reason,number_of_sprinkler_heads_operating,box
0,8028304,0,80283040,80920257,B06,11,1,4,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
135,3001842,0,30018420,30060286,B03,01,3,11,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
154,3003339,0,30033390,30110361,B07,31,2,9,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
160,3003851,0,30038510,30130312,B09,42,1,4,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
525,3001648,0,30016480,30050373,B09,43,1,4,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
595,3006544,0,30065440,30230200,B02,06,2,6,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
801,3000992,0,30009920,30030358,B10,17,1,4,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
830,3002518,0,30025180,30090011,B06,07,1,4,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
855,3004764,0,30047640,30160411,B08,19,2,6,0,0,...,-,-,-,-,False,-,-,-,0.0,0.0
